<a href="https://colab.research.google.com/github/minoring/Machine-Learning-Example/blob/master/Statoil_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 필요한 라이브러리 import

import pandas as pd 
import numpy as np 

In [2]:
# 파일 업로드

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minoring","key":"877d61d0cbb1e712d00118114ddcd82f"}'}

In [0]:
# kaggle.json 파일 이동

!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle

In [4]:
# 첼린지 불러오기
## 밑에 파일을 보면 .7z 파일이어서 압축을 풀어주어야 합니다.

!kaggle competitions download -c statoil-iceberg-classifier-challenge
!ls

  0% 0.00/37.7k [00:00<?, ?B/s]
100% 37.7k/37.7k [00:00<00:00, 32.6MB/s]
 96% 41.0M/42.9M [00:00<00:00, 63.5MB/s]
100% 42.9M/42.9M [00:00<00:00, 96.5MB/s]
 91% 224M/245M [00:02<00:00, 63.8MB/s]
100% 245M/245M [00:02<00:00, 86.7MB/s]
sample_data  sample_submission.csv.7z  test.json.7z  train.json.7z


In [5]:
# 7z 파일을 압축풀기

!apt-get install p7zip-full
!p7zip -d ./sample_submission.csv.7z
!p7zip -d ./train.json.7z
!p7zip -d ./test.json.7z

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 38566 bytes (38 KiB)

Extracting archive: ./sample_submission.csv.7z
--
Path = ./sample_submission.csv.7z
Type = 7z
Physical Size = 38566
Headers Size = 178
Method = LZMA2:17
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       117951
Compressed: 38566

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16

In [0]:
# 데이터 불러오기

train = pd.read_json('./data/processed/train.json')
test = pd.read_json('./data/processed/test.json')

In [7]:
# train.json 확인하기
## 데이터를 확인하면 band_1, band_2, id, inc_angle, is_iceberg가 있습니다.
### band_1은 HH, band_2는 HV 입니다. 레이더 방식에 관련된 내용인데 
# 정확한 내용은 모르고 이미지로 나타낼 수 있습니다.
### id는 이미지 id입니다.
### inc_angle은 레이더 입사각입니다.
### is_iceberg는 이미지에 나온 것이 빙산이면 1, 선박이면 0인 것을 나타낸 것입니다. (전문가가 판단한 것)

print("train =")
train.head(5)

train =


,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1
3,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",4cfc3a18,43.8306,0
4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",271f93f4,35.6256,0


In [8]:
print("Shape of train set:", train.shape)
print("Shape of test set:", test.shape)

Shape of train set: (1604, 5)
Shape of test set: (8424, 4)


In [10]:
print("Shape of band 1:", np.shape(train.band_1.iloc[0]))
print("Shape of band 2:", np.shape(train.band_2.iloc[0]))

Shape of band 1: (5625,)
Shape of band 2: (5625,)


In [12]:
# band1, band2 타입 확인하기
## 리스트로 되어있다고 합니다.
print("Type of band 1:",  type(train.band_1.iloc[0]))
print("Type of band 2:",  type(train.band_2.iloc[0]))



Type of band 1: <class 'list'>
Type of band 2: <class 'list'>


In [0]:
train[train['inc_angle'] == 'na'] = \
    train[train['inc_angle'] != 'na']['inc_angle'].mean()


In [0]:
import math

train['inc_angle'] = train['inc_angle'].apply(lambda x: math.radians(x))

In [19]:
train.inc_angle.head()

0    0.766617
1    0.665951
2    0.790388
3    0.764988
4    0.621784
Name: inc_angle, dtype: float64

In [0]:
def find_missing_data(series, shape):
  print('shape is ', shape)
  count = 0
  missing_list = []
  for i, x in enumerate(series):
    if np.shape(series.iloc[i]) != shape:
      missing_list.append(i)
      count += 1
      
  return missing_list, count

In [23]:
# train에 band_1에 missing data가 있는지 확인합니다.
## 총 133개가 나왔습니다.
missing_list1, count1 = find_missing_data(train.band_1, (5625,))
print("count: ", count1)
print("missing data: ", missing_list1)

shape is  (5625,)
count:  133
missing data:  [101, 126, 142, 145, 176, 179, 191, 278, 286, 342, 412, 431, 440, 493, 642, 666, 667, 730, 741, 742, 763, 822, 863, 868, 906, 916, 996, 1041, 1065, 1095, 1199, 1228, 1247, 1257, 1268, 1290, 1345, 1403, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584, 1585, 1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 1600, 1601, 1602, 1603]


In [24]:
# 진짜 missing data인지 확인해봅니다.
## 보면 100번째는 5625가 나오는데 101은 값이 없습니다.
print(np.shape(train.band_1.iloc[100]))
print(np.shape(train.band_1.iloc[101]))

(5625,)
()


In [25]:
# train에 band_2도 missing data가 있는지 확인해 봅니다.
## 똑같이 133개가 나옵니다.
missing_list2, count2 = find_missing_data(train.band_2, (5625,))
print("count: ", count1)
print("missing data: ", missing_list2)

shape is  (5625,)
count:  133
missing data:  [101, 126, 142, 145, 176, 179, 191, 278, 286, 342, 412, 431, 440, 493, 642, 666, 667, 730, 741, 742, 763, 822, 863, 868, 906, 916, 996, 1041, 1065, 1095, 1199, 1228, 1247, 1257, 1268, 1290, 1345, 1403, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533, 1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566, 1567, 1568, 1569, 1570, 1571, 1572, 1573, 1574, 1575, 1576, 1577, 1578, 1579, 1580, 1581, 1582, 1583, 1584, 1585, 1586, 1587, 1588, 1589, 1590, 1591, 1592, 1593, 1594, 1595, 1596, 1597, 1598, 1599, 1600, 1601, 1602, 1603]


In [26]:
# band_1과 band_2의 missing data가 똑같은지 확인해봅니다.
## 똑같다고 나옵니다.
missing_list1 == missing_list2

True

In [0]:
def drop_data(df, index):
  return df.drop(df.index[index])

In [0]:
train = drop_data(train, missing_list1)

In [30]:
train.shape

(1471, 5)

In [31]:
print(len(train[train['is_iceberg'] == 1.0]))
print(len(train[train['is_iceberg'] == 0.0]))

753
718


In [0]:
def standardise_vector(vector):
  standardised_vector = (np.array(vector) - np.mean(vector)) / np.std(vector)
  return standardised_vector.tolist()

In [0]:
train['band_1'] = train['band_1'].apply(standardise_vector)
train['band_2'] = train['band_2'].apply(standardise_vector)

In [35]:
# 표준화 해준 것을 확인합니다.
train.head(5)

,band_1,band_2,id,inc_angle,is_iceberg
0,"[0.01182174564467684, 0.27378275943768166, -0....","[1.1573585860406173, 0.15631457838356574, -0.4...",dfd5f913,0.766617,0.0
1,"[0.42137323598577087, -0.43078366534450846, -0...","[-2.0950953014724543, -0.8948057535299927, -0....",e25388fd,0.665951,0.0
2,"[-0.6969623073924855, -0.6969793944501068, -0....","[-0.014649839717716016, 0.35157292381049343, 1...",58b2aaa0,0.790388,1.0
3,"[0.2946591706447792, 0.04985985071106006, -0.3...","[0.7023040788844376, 0.8569610477684707, 1.005...",4cfc3a18,0.764988,0.0
4,"[-0.31533024179271146, 0.9175181439939705, 0.9...","[-0.693769690678262, -2.0346940986353044, -2.0...",271f93f4,0.621784,0.0


In [36]:
# reshape 예시입니다.
## numpy에서는 배열의 모양을 바꿀 수 있습니다.

A = np.array([[1, 2, 3], [4, 5, 6]])
print(A)
B = A.reshape((3, 2))
print(B)

[[1 2 3]
 [4 5 6]]
[[1 2]
 [3 4]
 [5 6]]


In [37]:
# concatenate 예제입니다.
## numpy에서는 여러 개의 배열을 한 개로 합칠 수 있습니다.
## axis = 0 은 Y축(세로방향)
## axis = 1 은 X축(가로방향)

A = np.array([[1, 2], [3, 4]])
B = np.array([[5, 6], [7, 8]])
C_Y = np.concatenate((A, B), axis = 0)
print(C_Y)
C_X = np.concatenate((A, B), axis = 1)
print(C_X)

[[1 2]
 [3 4]
 [5 6]
 [7 8]]
[[1 2 5 6]
 [3 4 7 8]]


In [50]:
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75)
                   for band in train['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75)
                   for band in train['band_2']])

print("x_band1[0]: ", x_band1[0])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis], 
                          x_band2[:, :, :, np.newaxis]],
                          axis=-1)

print("X_train[0]: ", X_train[0])
y_train = np.array(train["is_iceberg"])
print("Xtrain: ", X_train.shape)

x_band1[0]:  [[ 0.01182175  0.27378276 -0.27403224 ...  0.84555185  0.51450515
  -0.9390252 ]
 [-0.27400148  0.01180728  0.14550957 ...  0.51452035 -0.27509823
  -1.3322748 ]
 [-0.27400148  0.27378276  0.7400879  ...  0.27271712  0.010725
  -1.7818828 ]
 ...
 [-0.42750818 -0.27423045 -0.9381567  ... -0.75960714  0.73882335
   0.51429135]
 [-1.3313762   0.0115935  -0.27424565 ...  0.14424498  0.3957149
   0.14421389]
 [-0.42750818  0.39679757  0.2735545  ...  0.3957301   0.01051158
  -0.12916604]]
X_train[0]:  [[[ 0.01182175  1.1573585 ]
  [ 0.27378276  0.15631458]
  [-0.27403224 -0.4705372 ]
  ...
  [ 0.84555185  0.3421723 ]
  [ 0.51450515  0.15505895]
  [-0.9390252   1.0071673 ]]

 [[-0.27400148 -0.47050273]
  [ 0.01180728 -1.5198598 ]
  [ 0.14550957 -0.7059459 ]
  ...
  [ 0.51452035 -1.8384778 ]
  [-0.27509823  0.6898049 ]
  [-1.3322748   1.4367886 ]]

 [[-0.27400148 -1.5198405 ]
  [ 0.27378276 -2.9988198 ]
  [ 0.7400879  -0.7059459 ]
  ...
  [ 0.27271712 -0.7071675 ]
  [ 0.010725   

In [53]:
from keras.models import Sequential
from keras.layers import Conv2D, GlobalMaxPooling2D, Dense, Dropout, Input, Flatten, Activation, MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.optimizers import Adam

# convolution layer 혹은 fully connected layer를 쌓아 나가는 과정이라서 Sequential을 적어줍니다.
## flatten([N, 75, 75, 1] -> [N, 5625])을 하는 이유는 fully connected layer를 사용하기 위해서 입니다.
## 이미지 일때는 2D Convolution Layer를 사용합니다.
model = Sequential() # Sequential API를 사용하겠다고 선언합니다.
# Conv Layer 1
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(75, 75, 2))) # 첫 번째 Layer ## 첫 번째 Layer에는 input_shape을 적습니다. (필수는 아님)
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Dropout(0.2)) # 몇 퍼센트 비율로 제외하고 학습시킬 것인가를 나타냅니다.
# Conv Layer 2
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu")) # 두 번째 Layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.3))
# Flatten
model.add(Flatten())
# Dense Layer 1
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# Dense Layer 2
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))
# Sigmoid Layer         
model.add(Dense(1, activation="sigmoid"))
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
# gradient가 계산이 되었으면 gradient를 이용해서 weight를 업데이트 하는 부분이 'Optimizer'이고, 그 중에서 'Adam Optimizer'를 사용했습니다.
# metrics=["accuracy"]는 얼마나 잘 학습이 되었는지 확인하는 것입니다.
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 73, 73, 32)        608       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 18496)             0         
__________

In [54]:
model.fit(X_train, y_train, validation_split=0.2, epochs = 20)

Instructions for updating:
Use tf.cast instead.
Train on 1176 samples, validate on 295 samples
Epoch 1/20
1176/1176 [==============================] - 22s 18ms/step - loss: 1.4786 - acc: 0.4745 - val_loss: 0.6938 - val_acc: 0.4576
Epoch 2/20
1176/1176 [==============================] - 20s 17ms/step - loss: 0.6828 - acc: 0.5612 - val_loss: 0.6351 - val_acc: 0.6407
Epoch 3/20
1176/1176 [==============================] - 21s 18ms/step - loss: 0.6038 - acc: 0.6667 - val_loss: 0.5890 - val_acc: 0.6576
Epoch 4/20
1176/1176 [==============================] - 21s 18ms/step - loss: 0.5232 - acc: 0.7304 - val_loss: 0.4908 - val_acc: 0.7525
Epoch 5/20
1176/1176 [==============================] - 21s 18ms/step - loss: 0.4478 - acc: 0.7951 - val_loss: 0.4163 - val_acc: 0.8068
Epoch 6/20
1176/1176 [==============================] - 20s 17ms/step - loss: 0.4183 - acc: 0.8112 - val_loss: 0.4841 - val_acc: 0.7559
Epoch 7/20
1176/1176 [==============================] - 21s 18ms/step - loss: 0.3960 - ac

In [0]:
# 앞에 train에 해주었던 표준화를 test에도 해줍니다. (이하 생략)
test['inc_angle'] = test['inc_angle'].apply(lambda x: math.radians(x))

In [56]:
missing_list3, count3 = find_missing_data(test.band_1, (5625,))
print("count: ", count3)
print("missing data: ", missing_list3)

shape is  (5625,)
count:  0
missing data:  []


In [57]:
missing_list4, count4 = find_missing_data(test.band_2, (5625,))
print("count: ", count4)
print("missing data: ", missing_list4)


shape is  (5625,)
count:  0
missing data:  []


In [0]:
test['band_1'] = test['band_1'].apply(standardise_vector)
test['band_2'] = test['band_2'].apply(standardise_vector)

In [59]:
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
print("Xtest:", X_test.shape)

Xtest: (8424, 75, 75, 2)


In [60]:
prediction = model.predict(X_test, verbose=1)

8424/8424 [==============================] - 32s 4ms/step


In [0]:
submit = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.flatten()})
submit.to_csv("Submitted_first.csv", index=False)

In [64]:
!pwd


/content
